In [2]:
%pip install neo4j
from graphclass import BioAssayGraphDatabase
import pandas as pd
import requests
import json



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
def get_cids_from_sid(sid:int): 
    # Replace with the actual Substance ID

    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/sid/{sid}/cids/JSON"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        cids = data.get('InformationList', {}).get('Information', [{}])[0].get('CID', [])
        print(f"Substance ID: {sid}, Compound IDs: {cids}")
        return [True , cids]
    else:
        print(f"Failed to retrieve compound IDs for substance ID {sid}: {response.status_code}")
        return [False, -999]
def get_TypeSidAcitivity_FromAid(aid:int):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{aid}/JSON"

    # Make a GET request to the URL
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the JSON response
        assay_data = response.json()
        #print(json.dumps(assay_data, indent=4))
        # Iterate over the results and print the Substance ID and Activity Outcome
        #get all key of assay_data
        #print(assay_data['PC_AssaySubmit'])
        #print("______________________________ch____________________________")
        #print(json.dumps(assay_data['PC_AssaySubmit']['data'] , indent=4))
        return [True,assay_data['PC_AssaySubmit']['data']]
            
    else:
        print(f"Failed to retrieve assay results: {response.status_code}") 
        return [False, -999]
def GetResultJsonFromAid(aid:int):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{aid}/JSON"
    response = requests.get(url)
    if response.status_code == 200:
        assay_data = response.json()
        # print("______________________________ch____________________________")
        # print(json.dumps(assay_data['PC_AssaySubmit']['data'] , indent=4))
        return [True,assay_data['PC_AssaySubmit']['data'] ]
            
    else:
        print(f"Failed to retrieve assay results: {response.status_code}")
        return [False, -999]


In [4]:
# Replace 'your_file.csv' with the path to your CSV file
file_path_compound = 'data/compound.csv'
file_path_bioassay_bio = 'data/bioassay_assay.csv' 
 
# Read the CSV into a pandas data frame (df)
df_compound = pd.read_csv(file_path_compound, sep=',')
df_bioassay_bio = pd.read_csv(file_path_bioassay_bio, sep=',')

/var/folders/tw/ksffzqq93t56xq1d5v8_ldyc0000gn/T/ipykernel_1922/3131168568.py:6: DtypeWarning: Columns (32,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df_compound = pd.read_csv(file_path_compound, sep=',')
/var/folders/tw/ksffzqq93t56xq1d5v8_ldyc0000gn/T/ipykernel_1922/3131168568.py:7: DtypeWarning: Columns (16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bioassay_bio = pd.read_csv(file_path_bioassay_bio, sep=',')


In [5]:
# Usage Example:
uri = "bolt://localhost:7687"  # Replace with your Neo4j URI
username = "neo4j"  # Replace with your Neo4j username
password = "root12345"  # Replace with your Neo4j password

db = BioAssayGraphDatabase(uri, username, password)

In [6]:
# config to see all column in dataframe
pd.set_option('display.max_columns', None)

In [7]:
df_bioassay_bio.head()

,aid,aidtype,aidname,aiddesc,aidcomment,aidsrcid,aidsrcname,aidextid,depdate,aidmdate,cids,sids,geneids,aidcategories,protacxns,depcatg,pmids,rnai,ecs,repacxns,taxids,cellids,targettaxid,cnt,activecnt,anatomyid,anatomy,annotation
0,1764210,Literature-derived,Cytotoxicity against patient-derived human col...,"Title: Synthesis of novel 1H-Pyrazolo[3,4-b]py...",Journal: Bioorg Med Chem Lett_||_Year: 2021_||...,43,ChEMBL,CHEMBL4823208,20220830,20220830,164625371,469821412,NaN,Toxicity,NaN,Curation Efforts|Research and Development,34182093,0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,Colon cancer organoid,NaN
1,1259258,Screening,A screen to identify synergistic combinations ...,Kinome wide siRNA screen for synergistic combi...,NaN,94,"Cancer Functional Genomics, Wellcome Trust San...",COREAD_SIRNA_SECONDARY_SW48,20170330,20171011,NaN,56475149|56475150|56475471|56475472|56475473|5...,90|204|659|673|1021|1032|1108|1326|1459|2869|2...,Cell-based|RNAi,NaN,Legacy Depositors|Research and Development,28972570,1,NaN,NaN,9606,4182.0,NaN,160.0,40.0,NaN,NaN,NaN
2,1259259,Screening,A screen to identify synergistic combinations ...,Kinome wide siRNA screen for synergistic combi...,NaN,94,"Cancer Functional Genomics, Wellcome Trust San...",COREAD_SIRNA_PRIMARY_SW1463,20170330,20171011,NaN,56478465|56478467|56478468|56478474|56478482|5...,25|27|48|90|91|92|93|94|132|156|157|203|204|20...,Cell-based|RNAi,NaN,Legacy Depositors|Research and Development,28972570,1,NaN,NaN,9606,4176.0,NaN,794.0,9.0,NaN,NaN,NaN
3,1259260,Screening,A screen to identify synergistic combinations ...,Kinome wide siRNA screen for synergistic combi...,NaN,94,"Cancer Functional Genomics, Wellcome Trust San...",COREAD_SIRNA_PRIMARY_NCIH716,20170330,20171011,NaN,56478465|56478467|56478468|56478474|56478482|5...,25|27|48|90|91|92|93|94|132|156|157|203|204|20...,Cell-based|RNAi,NaN,Legacy Depositors|Research and Development,28972570,1,NaN,NaN,9606,4041.0,NaN,794.0,13.0,NaN,NaN,NaN
4,1259261,Screening,A screen to identify synergistic combinations ...,Kinome wide siRNA screen for synergistic combi...,NaN,94,"Cancer Functional Genomics, Wellcome Trust San...",COREAD_SIRNA_PRIMARY_HCC2998,20170330,20171011,NaN,56478465|56478467|56478468|56478474|56478482|5...,25|27|48|90|91|92|93|94|132|156|157|203|204|20...,Cell-based|RNAi,NaN,Legacy Depositors|Research and Development,28972570,1,NaN,NaN,9606,3729.0,NaN,794.0,10.0,NaN,NaN,NaN


In [8]:
#filter NAN on bioassay_bio  
df_bioassay_bio_CleanCids = df_bioassay_bio.dropna(subset=['cids']) 
df_bioassay_bio_CleanCids = df_bioassay_bio_CleanCids.dropna(subset=['sids'])


In [9]:
display(df_bioassay_bio_CleanCids.head())
print(df_bioassay_bio_CleanCids.shape) 
print(df_bioassay_bio.shape)

,aid,aidtype,aidname,aiddesc,aidcomment,aidsrcid,aidsrcname,aidextid,depdate,aidmdate,cids,sids,geneids,aidcategories,protacxns,depcatg,pmids,rnai,ecs,repacxns,taxids,cellids,targettaxid,cnt,activecnt,anatomyid,anatomy,annotation
0,1764210,Literature-derived,Cytotoxicity against patient-derived human col...,"Title: Synthesis of novel 1H-Pyrazolo[3,4-b]py...",Journal: Bioorg Med Chem Lett_||_Year: 2021_||...,43,ChEMBL,CHEMBL4823208,20220830,20220830,164625371,469821412,NaN,Toxicity,NaN,Curation Efforts|Research and Development,34182093,0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,Colon cancer organoid,NaN
52,1733006,Confirmatory,Growth inhibition of human colon cancer cell l...,"Title: Discovery of 4-alkoxy-2-aryl-6,7-dimeth...",Compounds with activity <= 10uM or explicitly ...,43,ChEMBL,CHEMBL4738934,20220318,20220830,162646831|162650206|162664933,461519887|461524749|461545721,NaN,NaN,NaN,Curation Efforts|Research and Development,33631697,0,NaN,NaN,NaN,129058.0,NaN,3.0,2.0,NaN,Colon,NaN
53,1820036,Literature-derived,Anticancer activity against KOES-007 patient d...,"Title: Design, Synthesis, and Biological Evalu...",Journal: J Med Chem_||_Year: 2021_||_Volume: 6...,43,ChEMBL,CHEMBL5042739,20230314,20230314,166631882,475991183,NaN,NaN,NaN,Curation Efforts|Research and Development,33876637,0,NaN,NaN,9606,NaN,NaN,1.0,1.0,NaN,Stomach cancer organoid,NaN
54,1820038,Literature-derived,Anticancer activity against KOES-007 patient d...,"Title: Design, Synthesis, and Biological Evalu...",Journal: J Med Chem_||_Year: 2021_||_Volume: 6...,43,ChEMBL,CHEMBL5042741,20230314,20230314,166631882,475991183,NaN,NaN,NaN,Curation Efforts|Research and Development,33876637,0,NaN,NaN,9606,NaN,NaN,1.0,NaN,NaN,Stomach cancer organoid,NaN
55,1896227,Confirmatory,Antitumor activity against patient-derived Lun...,Title: Targeting Extracellular Signal-Regulate...,Compounds with activity <= 10uM or explicitly ...,43,ChEMBL,CHEMBL5152489,20230629,20230629,90331177,482060909,NaN,NaN,NaN,Curation Efforts|Research and Development,36205714,0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN


(288951, 28)
(337730, 28)


In [14]:
df_bioassay_bio_CleanAid = df_bioassay_bio_CleanCids.dropna(subset=['aidcategories'])

In [19]:
display(df_bioassay_bio_CleanAid.head())

,aid,aidtype,aidname,aiddesc,aidcomment,aidsrcid,aidsrcname,aidextid,depdate,aidmdate,cids,sids,geneids,aidcategories,protacxns,depcatg,pmids,rnai,ecs,repacxns,taxids,cellids,targettaxid,cnt,activecnt,anatomyid,anatomy,annotation
0,1764210,Literature-derived,Cytotoxicity against patient-derived human col...,"Title: Synthesis of novel 1H-Pyrazolo[3,4-b]py...",Journal: Bioorg Med Chem Lett_||_Year: 2021_||...,43,ChEMBL,CHEMBL4823208,20220830,20220830,164625371,469821412,NaN,Toxicity,NaN,Curation Efforts|Research and Development,34182093,0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,Colon cancer organoid,NaN
65,195297,Confirmatory,In vitro cytotoxicity against renal cancer cells.,Title: Design of cancer-specific antitumor age...,Compounds with activity <= 10uM or explicitly ...,43,ChEMBL,CHEMBL801845,20100524,20220830,9796346|9813608|9902599|9943807|10445119|24839556,103345284|103345511|103345939|103346008|103382...,NaN,Toxicity|In vitro,NaN,Curation Efforts|Research and Development,10669573,0,NaN,NaN,NaN,128960.0,NaN,6.0,1.0,NaN,NaN,NaN
96,1524334,Confirmatory,Cytotoxicity against human renal cancer cells ...,"Title: Synthesis, in vitro and in vivo biologi...",Compounds with activity <= 10uM or explicitly ...,43,ChEMBL,CHEMBL4320825,20210302,20220830,155513016|155516965|155518252|155523011|155526...,440113526|440119463|440121421|440128901|440134...,NaN,Toxicity,NaN,Curation Efforts|Research and Development,30784885,0,NaN,NaN,NaN,128960.0,NaN,15.0,11.0,NaN,NaN,NaN
112,214916,Confirmatory,In vitro median growth inhibitory concentratio...,Title: Synthesis and biological evaluation of ...,Journal: J. Med. Chem._||_Year: 2000_||_Volume...,43,ChEMBL,CHEMBL817971,20100524,20220830,5468388|5470910|5470913|5470914|5470917|547092...,103314296|103314350|103314374|103314378|103314...,NaN,In vitro,NaN,Curation Efforts|Research and Development,10956199,0,NaN,NaN,NaN,129122.0,NaN,8.0,NaN,NaN,NaN,NaN
113,214920,Confirmatory,In vitro median growth inhibitory concentratio...,Title: Synthesis and biological evaluation of ...,Compounds with activity <= 10uM or explicitly ...,43,ChEMBL,CHEMBL816268,20100524,20220830,5468388|5470906|5470910|5470913|5470914|547091...,103314296|103314350|103314352|103314374|103314...,NaN,In vitro,NaN,Curation Efforts|Research and Development,10956199,0,NaN,NaN,NaN,129060.0,NaN,9.0,1.0,NaN,NaN,NaN


In [17]:
#print unique data in 'aidcategories' column 
print(df_bioassay_bio_CleanAid['aidcategories'].unique())

['Toxicity' 'Toxicity|In vitro' 'In vitro' 'Cell-based|Toxicity'
 'Cell-based' 'RNAi' 'In vivo' 'Biochemical' 'Biochemical|In vitro'
 'Organism-based' 'Biochemical|Cell-based' 'Biochemical|Toxicity'
 'Organism-based|Toxicity' 'Biochemical|Cell-based|In vivo'
 'Toxicity|In vivo' 'Biochemical|In vivo' 'Organism-based|In vivo']


In [55]:
x = df_bioassay_bio_CleanAid[['aid','geneids','protacxns']]

x = x.dropna(subset=['geneids'])
x = x.dropna(subset=['protacxns'])
display(x)


,aid,geneids,protacxns
127,624396,598|4170,AAI07736|CAA80661
320,902,7157,NP_000537
321,903,7157,NP_000537
322,904,7157,NP_000537
323,924,7157,NP_000537
...,...,...,...
335521,1900760,60351,Q62735
335522,1900761,60351,Q62735
337145,1911306,10419,O14744
337176,1911337,10419,O14744


In [23]:
df_sids_cid= df_bioassay_bio_CleanAid[['aid','sids','cids','aidcategories']]
display(df_sids_cid.head())
print(df_sids_cid.shape)

,aid,sids,cids,aidcategories
0,1764210,469821412,164625371,Toxicity
65,195297,103345284|103345511|103345939|103346008|103382...,9796346|9813608|9902599|9943807|10445119|24839556,Toxicity|In vitro
96,1524334,440113526|440119463|440121421|440128901|440134...,155513016|155516965|155518252|155523011|155526...,Toxicity
112,214916,103314296|103314350|103314374|103314378|103314...,5468388|5470910|5470913|5470914|5470917|547092...,In vitro
113,214920,103314296|103314350|103314352|103314374|103314...,5468388|5470906|5470910|5470913|5470914|547091...,In vitro


(49306, 4)


In [42]:
#type of aidcategories
print(type(df_sids_cid['aidcategories'][0]))

<class 'str'>


In [30]:
db.create_node("Node1", "Label1", attribute1="value1")

In [ ]:
import logging

logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')
CompoundNode = set()
AidNode = set()

# Using itertuples for efficient iteration
for row in df_sids_cid.itertuples(index=False):
    cids = [int(cid) for cid in row.cids.split('|') if cid.isnumeric()]
    
    if row.aid not in AidNode:
        AidNode.add(row.aid)
        try:
            # Batch node creation if possible or create nodes as needed
            db.create_node(int(row.aid), "Bioassay", aidcategories=row.aidcategories)
        except Exception as e:
            logging.error(f"Error creating node for aid {row.aid}: {e}")
        
    logging.info(row.aidcategories)
    
    for cid in cids:
        if cid not in CompoundNode:
            CompoundNode.add(cid)
            try:
                # Batch node creation if possible or create nodes as needed
                db.create_node(int(cid), "Compound")
            except Exception as e:
                logging.error(f"Error creating node for cid {cid}: {e}")
        
        try:
            # Batch relationship creation if possible or create relationships as needed
            db.create_relationship(cid, str(row.aidcategories), row.aid)
        except Exception as e:
            logging.error(f"Error creating relationship between cid {cid} and aid {row.aid}: {e}")

logging.info(len(CompoundNode))


In [41]:
#delete all node in neo4j cypher query 
#MATCH (n) DETACH DELETE n


# MATCH (c:Compound {name: 14413056})-->(b:Bioassay)
# RETURN c,b; 

#cypher to get number of all bioassay lable node 
#MATCH (n:Bioassay) RETURN count(n): 
